In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

In [ ]:
games=pd.read_csv("steam_games.csv")

data understanding

In [ ]:
games.head()

In [ ]:
games.tail()

In [ ]:
games.shape

In [ ]:
games.describe()

In [ ]:
games.info()

Data Preparation

In [ ]:
#ELIMINAR COLUMNAS QUE NO SIRVEN PARA EL ANALISIS
#la columna title contiene la misma informacion que app
games.drop(columns=["title"], inplace=True)
#la informacion en las url no es importante
games.drop(columns=["url", "reviews_url","tags"], inplace=True)
#eliminar las filas que tienen mas de 4 nan
games.dropna(thresh=4, inplace=True)
#la columna price no se usa para nada en esta ocasion
games.drop(columns=["price"], inplace=True)

In [ ]:
#CAMBIAR LOS DATOS DENTRO DE LAS COLUMNAS
#cambio la columna de id a string
games["id"]=games["id"].astype(str)

In [ ]:
#la columna de date time solo debe tener fechas y no horas
games['release_date'] = games['release_date'].dt.date

In [ ]:
#ELIMINAR LA PUNTUACION
import string
games["publisher"] = games["publisher"].str.replace('[{}]'.format(string.punctuation), '',regex=True)
games["app_name"] = games["app_name"].str.replace('[{}]'.format(string.punctuation), '',regex=True)
games["developer"] = games["developer"].str.replace('[{}]'.format(string.punctuation), '',regex=True)

In [ ]:
games = games.drop_duplicates(keep="first")

In [ ]:
#completar los nan

games["genres"].fillna((games["genres"].mode()[0]),inplace=True)
games["release_date"].fillna((games["release_date"].mode()[0]),inplace=True)
games["publisher"].fillna(method="ffill",inplace=True)
games["specs"].fillna((games["specs"].mode()[0]),inplace=True)
games["id"].fillna(method="ffill",inplace=True)
games["developer"].fillna((games["developer"].mode()[0]),inplace=True)

In [ ]:
#CAMBIAR LAS COLUMNAS DE GENERO Y SPECS
#definir una funcion para cambiar a listas los datos de las columnas
import ast
def tolist(lst):
    if isinstance(lst, str) :
        lst = lst.replace("0's",'0s')
        lst = ast.literal_eval(lst)

    return list(lst)

In [ ]:
#convertir a listas
games['genres'] = games['genres'].apply(lambda x : tolist(x))
games['specs'] = games['specs'].apply(lambda x : tolist(x))

In [ ]:
# crear las columnas binarias de los géneros
def create_genre_columns(dataframe):
    genres_list = set()

    for genres in dataframe['genres']:
        if isinstance(genres, list):
            genres_list.update(genres)
    
    # crea una columna por cada genre en genre_list 
    for genre in genres_list:
        dataframe["genre_" + genre] = dataframe['genres'].apply(lambda x: 1 if genre in x else 0)
    
    return dataframe

create_genre_columns(games)


In [ ]:
# misma funcion para la columna specs
def create_specs_columns(dataframe):
    specs_list = set()

    for specs in dataframe['specs']:
        if isinstance(specs, list):
            specs_list.update(specs)
    
    for specs in specs_list:
        dataframe["specs_" + specs] = dataframe['specs'].apply(lambda x: 1 if specs in x else 0)
    
    return dataframe

create_specs_columns(games)

In [ ]:
#eliminar las columnas originales
games.drop(columns=["genres", "specs"], inplace=True)

In [ ]:
#fecha frecuencias
ax=games["release_date"].value_counts() \
    .head(10) \
    .plot(kind="bar", title="top de fechas con mas lanzamientos")
ax.set_xlabel("Fecha de lanzamiento")
ax.set_ylabel("cuenta")

In [ ]:
#developer frecuencias
ax=games["developer"].value_counts() \
    .head(10) \
    .plot(kind="bar", title="top de developers")
ax.set_xlabel("nombre developer")
ax.set_ylabel("cuenta")

In [ ]:
#publisher frecuencias
ax=games["publisher"].value_counts() \
    .head(10) \
    .plot(kind="bar", title="top de publisher")
ax.set_xlabel("nombre publisher")
ax.set_ylabel("cuenta")